# Crawling 크롤링 + 전처리

In [1]:
# 웹 사이트로부터 데이터를 얻어오는 클래스
import requests
import json

class Requester:
    def get_data(self, url, parameters = {}):
        if (url is None) or (url.strip() is ''):
            raise Exception('url should not be empty')
        
        response = requests.get(url.strip(), params = parameters)
        if response.status_code is not 200:
            print('[ERROER]Can not receive response')

        response_text = (response.text is None) and '{}' or response.text
        json_response = json.loads(response_text)
        return json_response

## '카테고리 목록'을 크롤링하는 클래스

In [2]:
# 홈페이지 주소: https://ohou.se/store/category?category=0_1_0_7
class CategoryCrawler:
    def __init__(self):
        self.categories = list()
        self.requester = Requester()
    
    def collect_all_sub_categories(self, sub_categories):
        if (sub_categories is None) or (len(sub_categories) <= 0):
            return []
        
        collected_sub_categories = list()
        for inner_sub_category in sub_categories:
            if (inner_sub_category['child'] is None) or (len(inner_sub_category['child']) <= 0):
                collected_sub_categories.append({
                    'category_name': inner_sub_category['title'].strip(),
                    'category_key': inner_sub_category['hash'].strip()
                })
                continue
            found_sub_categories = self.collect_all_sub_categories(inner_sub_category['child'])
            collected_sub_categories = collected_sub_categories + found_sub_categories
            
        return collected_sub_categories
    
    def collect(self):
        self.categories = list()
        parameters = {'used': str(2), 'include_html': 'true'}
        url = 'https://ohou.se/productions/categories.json'
        response = self.requester.get_data(url, parameters)
        
        for category in response['categories']:
            item = {
                'category_name': category['title'].strip(),
                'category_key': category['hash'].strip(),
                'sub_categories': [],
            }
            item['sub_categories'] = self.collect_all_sub_categories(category['child'])
            self.categories.append(item)

In [3]:
category_crawler = CategoryCrawler()
category_crawler.collect()

print('> Total main category count:', len(category_crawler.categories))
print('> "{}"\'s sub categories: {}'.format(
    category_crawler.categories[4]['category_name'],
    category_crawler.categories[4]['sub_categories']
))

> Total main category count: 12
> "수납/정리"'s sub categories: [{'category_name': '플라스틱서랍장', 'category_key': '11_0_0'}, {'category_name': '이동식정리함/트롤리', 'category_key': '11_0_2'}, {'category_name': '공간박스', 'category_key': '11_0_3'}, {'category_name': '리빙박스', 'category_key': '11_1_0'}, {'category_name': '수납정리함', 'category_key': '11_1_1'}, {'category_name': '기타수납/정리용품', 'category_key': '11_1_2'}, {'category_name': '수납바스켓', 'category_key': '11_2_0'}, {'category_name': '라탄바스켓', 'category_key': '11_2_1'}, {'category_name': '빨래바구니/보관함', 'category_key': '11_2_2'}, {'category_name': '소품트레이', 'category_key': '11_2_3'}, {'category_name': '스탠드행거', 'category_key': '11_3_0'}, {'category_name': '이동식행거', 'category_key': '11_3_1'}, {'category_name': '고정식행거', 'category_key': '11_3_2'}, {'category_name': '벽선반', 'category_key': '11_4_0'}, {'category_name': '스탠드선반', 'category_key': '11_4_1'}, {'category_name': '세탁기선반', 'category_key': '11_4_2'}, {'category_name': '옷걸이/바지걸이', 'category_key': '11_5_0'}, {'categ

## '카테고리별' 모든 '상품' 및 관련된 '리뷰텍스트 등'을 크롤링하는 클래스

In [4]:
from time import sleep

class ProductCrawler:
    def __init__(self, categories = []):
        self.products = list()
        self.categories = categories
        self.requester = Requester()
        self.default_page_size = 1000 # 1페이지에 출력될 아이템 갯수

    def calculat_total_page(self, total_item_count):
        if (total_item_count is None) or (total_item_count <= 0):
            return 1
        total_page = (total_item_count // self.default_page_size)
        if (total_item_count % self.default_page_size) is not 0:
            total_page = total_page + 1
        return total_page
    
    def get_production_data_by_page(self, category_key, page):
        parameters = {
             'v': str(2), # 고정 queryParam 값. v인걸 보아 API 버전 정보인듯?
             'per': str(self.default_page_size), # 고정 queryParam 값. 페이에 출력될 아이템 갯수
             'category': category_key,
             'page': str(page)
        }
        url = 'https://ohou.se/store/category.json'
        response = self.requester.get_data(url, parameters)
        return response
    
    def collect_productions(self, productions = []):
        for production in productions:
            self.products.append({
                'id': production['id'],
                'name': production['name'],
                'review_count': production['review_count'], #  제품에 남겨진 리뷰 총 갯수
                'review_average': production['review_avg'],  # 제품에 남겨진 평점 평균
                'scrap_count': production['scrap_count'],  # 제품을 스크랩한 총 갯수
                'view_count': production['view_count'],  # 제품을 본 횟수
                #'review_text': '' # 이 부분은 아래 코드 블록에서 수행 예정
            })
        
    def collect(self):
        self.products = list()
        if (self.categories is None) or (len(self.categories) <= 0):
            return
        
        for category in self.categories:
            category_keys = [sub['category_key'] for sub in category['sub_categories']]
            for category_key in category_keys:
                print('> Start to collect productions. category_key:', category_key)
                current_page = 1
                response = self.get_production_data_by_page(category_key, current_page)
                if (response['productions'] is not None) and (len(response['productions']) > 0):
                    self.collect_productions(response['productions'])
                
                total_page = self.calculat_total_page(response['item_count']) # item_count: 현재 카테고리의 모든 제품 갯수
                print('>> This category_key has total_page:', total_page)
                if current_page is total_page:
                    break
                else:
                    while (current_page <= total_page):
                        current_page = current_page + 1
                        print('>> Collect proudctions at page:', current_page)
                        response = self.get_production_data_by_page(category_key, current_page)
                        self.collect_productions(response['productions'])
                    sleep(3)

In [5]:
product_crawler = ProductCrawler(categories = category_crawler.categories)
product_crawler.collect()

> Start to collect productions. category_key: 0_1_0_3
>> This category_key has total_page: 6
>> Collect proudctions at page: 2
>> Collect proudctions at page: 3
>> Collect proudctions at page: 4
>> Collect proudctions at page: 5
>> Collect proudctions at page: 6
>> Collect proudctions at page: 7
> Start to collect productions. category_key: 0_1_0_4
>> This category_key has total_page: 1
> Start to collect productions. category_key: 1_0_20
>> This category_key has total_page: 4
>> Collect proudctions at page: 2
>> Collect proudctions at page: 3
>> Collect proudctions at page: 4
>> Collect proudctions at page: 5
> Start to collect productions. category_key: 1_0_1
>> This category_key has total_page: 4
>> Collect proudctions at page: 2
>> Collect proudctions at page: 3
>> Collect proudctions at page: 4
>> Collect proudctions at page: 5
> Start to collect productions. category_key: 1_0_0
>> This category_key has total_page: 2
>> Collect proudctions at page: 2
>> Collect proudctions at page

In [6]:
print('Total product count:', len(product_crawler.products))
print(product_crawler.products[0])

Total product count: 25031
{'id': 448894, 'name': ' [단독] POL 아쿠아텍스 3인소파 2colors(스툴무료증정) ', 'review_count': 833, 'review_average': 4.51, 'scrap_count': 13467, 'view_count': 178668}


In [7]:
total_user_review_count = 0
for product in product_crawler.products:
    total_user_review_count = total_user_review_count + product['review_count']
print('Total user review count:', total_user_review_count)

Total user review count: 369033


## 'Product에 남겨진 리뷰텍스트'를 크롤링하는 클래스

In [8]:
# https://ohou.se/production_reviews.json?production_id=81625&page=2&order=best
from time import sleep

class ProductReviewCrawler:
    def __init__(self, products = []):
        self.products = products
        self.product_n_reviews = list()
        self.requester = Requester()
        self.default_page_size = 5000
    
    def calculat_total_page(self, total_item_count):
        if (total_item_count is None) or (total_item_count <= 0):
            return 1
        total_page = (total_item_count // self.default_page_size)
        if (total_item_count % self.default_page_size) is not 0:
            total_page = total_page + 1
        return total_page
    
    def get_production_review_data_by_page(self, product_id, page):
        parameters = {
             'order': 'best', # 고정 queryParam 값. 베스트 리뷰순으로 가져오기위한 정렬값
             'per': str(self.default_page_size), # 고정 queryParam 값. 페이에 출력될 아이템 갯수
             'production_id': str(product_id),
             'page': str(page)
        }
        url = 'https://ohou.se/production_reviews.json'
        response = self.requester.get_data(url, parameters)
        return response
    
    def collect_product_review(self, product_review_template, reviews):
        for review in reviews:
            new_product_n_review = product_review_template.copy()
            product_review_template['review_user_id'] = review['writer_id']
            product_review_template['review_status'] = review['status']
            product_review_template['review_text'] = review['review']['comment']
            product_review_template['review_star_durability'] = review['review']['star_durability']
            product_review_template['review_star_design'] = review['review']['star_design']
            product_review_template['review_star_cost'] = review['review']['star_cost']
            product_review_template['review_star_delivery'] = review['review']['star_delivery']
            product_review_template['review_star_avg'] = review['review']['star_avg']
            product_review_template['product_info_id'] = review['production_information']['id']
            product_review_template['production_info_name'] = review['production_information']['name']
            product_review_template['production_info_brand_name'] = review['production_information']['brand_name']
            product_review_template['production_info_explain'] = review['production_information']['explain']
            product_review_template['production_info_is_purchased'] = review['production_information']['is_purchased']
            
            self.product_n_reviews.append(new_product_n_review)
        
    def collect(self):
        self.product_n_reviews = list()
        if (self.products is None) or (len(self.products) <= 0):
            return
        
        for product in self.products:
            product_id = product['id']
            
            print('> Start to collect production reviews. product_id:', product_id)
            current_page = 1
            response = self.get_production_review_data_by_page(product_id, current_page)
            if (response['reviews'] is not None) and (len(response['reviews']) > 0):
                self.collect_product_review(product, response['reviews'])

            total_page = self.calculat_total_page(response['count']) # count: 현재 제품의 모든 리뷰 갯수
            
            print('>> This product has total_page:', total_page)
            if current_page is total_page:
                continue

            while (current_page <= total_page):
                current_page = current_page + 1
                print('>> Collect reviews at page:', current_page)
                response = self.get_production_review_data_by_page(product_id, current_page)
                if (response['reviews'] is not None) and (len(response['reviews']) > 0):
                    self.collect_product_review(product, response['reviews'])
                sleep(3)

In [9]:
product_review_crawler = ProductReviewCrawler(products = product_crawler.products)
product_review_crawler.collect()

> Start to collect production reviews. product_id: 448894
>> This product has total_page: 1
> Start to collect production reviews. product_id: 267218
>> This product has total_page: 1
> Start to collect production reviews. product_id: 384630
>> This product has total_page: 1
> Start to collect production reviews. product_id: 158748
>> This product has total_page: 1
> Start to collect production reviews. product_id: 205178
>> This product has total_page: 1
> Start to collect production reviews. product_id: 384634
>> This product has total_page: 1
> Start to collect production reviews. product_id: 173068
>> This product has total_page: 1
> Start to collect production reviews. product_id: 32151
>> This product has total_page: 1
> Start to collect production reviews. product_id: 361775
>> This product has total_page: 1
> Start to collect production reviews. product_id: 430554
>> This product has total_page: 1
> Start to collect production reviews. product_id: 365562
>> This product has tot

>> This product has total_page: 1
> Start to collect production reviews. product_id: 300500
>> This product has total_page: 1
> Start to collect production reviews. product_id: 144966
>> This product has total_page: 1
> Start to collect production reviews. product_id: 126537
>> This product has total_page: 1
> Start to collect production reviews. product_id: 147727
>> This product has total_page: 1
> Start to collect production reviews. product_id: 376489
>> This product has total_page: 1
> Start to collect production reviews. product_id: 442149
>> This product has total_page: 1
> Start to collect production reviews. product_id: 415061
>> This product has total_page: 1
> Start to collect production reviews. product_id: 542373
>> This product has total_page: 1
> Start to collect production reviews. product_id: 371965
>> This product has total_page: 1
> Start to collect production reviews. product_id: 347797
>> This product has total_page: 1
> Start to collect production reviews. product

>> This product has total_page: 1
> Start to collect production reviews. product_id: 359710
>> This product has total_page: 1
> Start to collect production reviews. product_id: 381058
>> This product has total_page: 1
> Start to collect production reviews. product_id: 268010
>> This product has total_page: 1
> Start to collect production reviews. product_id: 268120
>> This product has total_page: 1
> Start to collect production reviews. product_id: 86603
>> This product has total_page: 1
> Start to collect production reviews. product_id: 371492
>> This product has total_page: 1
> Start to collect production reviews. product_id: 37333
>> This product has total_page: 1
> Start to collect production reviews. product_id: 418047
>> This product has total_page: 1
> Start to collect production reviews. product_id: 226198
>> This product has total_page: 1
> Start to collect production reviews. product_id: 191900
>> This product has total_page: 1
> Start to collect production reviews. product_i

>> This product has total_page: 1
> Start to collect production reviews. product_id: 397747
>> This product has total_page: 1
> Start to collect production reviews. product_id: 239522
>> This product has total_page: 1
> Start to collect production reviews. product_id: 361337
>> This product has total_page: 1
> Start to collect production reviews. product_id: 361797
>> This product has total_page: 1
> Start to collect production reviews. product_id: 157754
>> This product has total_page: 1
> Start to collect production reviews. product_id: 217140
>> This product has total_page: 1
> Start to collect production reviews. product_id: 416008
>> This product has total_page: 1
> Start to collect production reviews. product_id: 368480
>> This product has total_page: 1
> Start to collect production reviews. product_id: 178062
>> This product has total_page: 1
> Start to collect production reviews. product_id: 307866
>> This product has total_page: 1
> Start to collect production reviews. product

>> This product has total_page: 1
> Start to collect production reviews. product_id: 257822
>> This product has total_page: 1
> Start to collect production reviews. product_id: 281184
>> This product has total_page: 1
> Start to collect production reviews. product_id: 484272
>> This product has total_page: 1
> Start to collect production reviews. product_id: 505316
>> This product has total_page: 1
> Start to collect production reviews. product_id: 380390
>> This product has total_page: 1
> Start to collect production reviews. product_id: 161166
>> This product has total_page: 1
> Start to collect production reviews. product_id: 520108
>> This product has total_page: 1
> Start to collect production reviews. product_id: 307208
>> This product has total_page: 1
> Start to collect production reviews. product_id: 242404
>> This product has total_page: 1
> Start to collect production reviews. product_id: 264834
>> This product has total_page: 1
> Start to collect production reviews. product

...

>> This product has total_page: 1
> Start to collect production reviews. product_id: 104301
>> This product has total_page: 1
> Start to collect production reviews. product_id: 104261
>> This product has total_page: 1
> Start to collect production reviews. product_id: 104259
>> This product has total_page: 1
> Start to collect production reviews. product_id: 104257
>> This product has total_page: 1
> Start to collect production reviews. product_id: 104254
>> This product has total_page: 1
> Start to collect production reviews. product_id: 104200
>> This product has total_page: 1
> Start to collect production reviews. product_id: 103499
>> This product has total_page: 1
> Start to collect production reviews. product_id: 103496
>> This product has total_page: 1
> Start to collect production reviews. product_id: 103492
>> This product has total_page: 1
> Start to collect production reviews. product_id: 103486
>> This product has total_page: 1
> Start to collect production reviews. product

>> This product has total_page: 1
> Start to collect production reviews. product_id: 85455
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85427
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85410
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85401
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85395
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85391
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85386
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85385
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85372
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85363
>> This product has total_page: 1
> Start to collect production reviews. product_id: 85360

>> This product has total_page: 1
> Start to collect production reviews. product_id: 52762
>> This product has total_page: 1
> Start to collect production reviews. product_id: 52760
>> This product has total_page: 1
> Start to collect production reviews. product_id: 51822
>> This product has total_page: 1
> Start to collect production reviews. product_id: 51171
>> This product has total_page: 1
> Start to collect production reviews. product_id: 50663
>> This product has total_page: 1
> Start to collect production reviews. product_id: 48656
>> This product has total_page: 1
> Start to collect production reviews. product_id: 48655
>> This product has total_page: 1
> Start to collect production reviews. product_id: 47297
>> This product has total_page: 1
> Start to collect production reviews. product_id: 47295
>> This product has total_page: 1
> Start to collect production reviews. product_id: 47288
>> This product has total_page: 1
> Start to collect production reviews. product_id: 47287

>> This product has total_page: 1
> Start to collect production reviews. product_id: 193331
>> This product has total_page: 1
> Start to collect production reviews. product_id: 192606
>> This product has total_page: 1
> Start to collect production reviews. product_id: 285576
>> This product has total_page: 1
> Start to collect production reviews. product_id: 216307
>> This product has total_page: 1
> Start to collect production reviews. product_id: 434359
>> This product has total_page: 1
> Start to collect production reviews. product_id: 129837
>> This product has total_page: 1
> Start to collect production reviews. product_id: 588758
>> This product has total_page: 1
> Start to collect production reviews. product_id: 308720
>> This product has total_page: 1
> Start to collect production reviews. product_id: 489888
>> This product has total_page: 1
> Start to collect production reviews. product_id: 562883
>> This product has total_page: 1
> Start to collect production reviews. product

KeyError: 'reviews'

In [10]:
print('Total product review count:', len(product_review_crawler.product_n_reviews))
print(product_review_crawler.product_n_reviews[0])
print(product_review_crawler.product_n_reviews[9000])

Total product review count: 56165
{'id': 448894, 'name': ' [단독] POL 아쿠아텍스 3인소파 2colors(스툴무료증정) ', 'review_count': 833, 'review_average': 4.51, 'scrap_count': 13467, 'view_count': 178668}
{'id': 32151, 'name': '[할인쿠폰] 타미 1인 패브릭소파 3colors', 'review_count': 4114, 'review_average': 4.56, 'scrap_count': 100895, 'view_count': 1708985, 'review_user_id': 5956503, 'review_status': '확인완료', 'review_text': '진짜 푹신하고, 생각보다 커서 다리 뻗을 수 잇고 너무 좋습니다ㅠㅠㅠ 최고!', 'review_star_durability': 5, 'review_star_design': 5, 'review_star_cost': 5, 'review_star_delivery': 5, 'review_star_avg': 5.0, 'product_info_id': 32151, 'production_info_name': '[할인쿠폰] 타미 1인 패브릭소파 3colors', 'production_info_brand_name': '보니애가구', 'production_info_explain': '아이보리', 'production_info_is_purchased': True}


# `input/data-product-review-v1.csv` 생성

In [ ]:
import os
import shutil

input_dir_path = os.path.join(os.getcwd(), 'input')
if os.path.exists(input_dir_path) is True:
    # 현재 위치에 input 폴더가 있으면 삭제
    shutil.rmtree(input_dir_path)

# 현재 위치에 input 폴더 생성
os.mkdir(input_dir_path)

In [ ]:
# input 폴더 만들고 data-product-review-v1.csv 에 product_n_reviews 내용 저장하기
import pandas

df = pandas.DataFrame(product_review_crawler.product_n_reviews)
df.head(3)

In [ ]:
df.to_csv("./input/data-product-review-v1.csv", sep='\t', na_rep='', encoding="UTF-16")

In [ ]:
# 잘 만들어졌는지 확인
pandas.read_csv('./input/data-product-review-v1.csv', sep='\t', encoding="UTF-16")